# TGWeb Mobile Android Build (Google Colab)
Минимальный ноутбук: данные по шагам, сборка отдельным финальным шагом.


In [ ]:
import subprocess

def sh(cmd: str):
    subprocess.run(["bash", "-lc", cmd], check=True)


In [ ]:
# STEP 1: install base packages
sh(r'''
set -euxo pipefail
export DEBIAN_FRONTEND=noninteractive
apt-get update
apt-get install -y --no-install-recommends   ca-certificates git curl unzip zip openjdk-17-jdk-headless
''')


In [ ]:
# STEP 2: clone project via HTTPS
# Public repo clone (no SSH key required)
sh(r'''
set -euxo pipefail
cd /content
rm -rf tgwebmobile
git clone https://github.com/Perdonus/tgwebmobile.git
''')


In [ ]:
# STEP 3: install Android cmdline-tools
sh(r'''
set -euxo pipefail
export ANDROID_SDK_ROOT=/content/android-sdk
mkdir -p "$ANDROID_SDK_ROOT/cmdline-tools"
curl -fL --retry 5 --retry-delay 5   https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip   -o /tmp/cmdline-tools.zip
unzip -q /tmp/cmdline-tools.zip -d "$ANDROID_SDK_ROOT/cmdline-tools"
rm -rf "$ANDROID_SDK_ROOT/cmdline-tools/latest"
mv "$ANDROID_SDK_ROOT/cmdline-tools/cmdline-tools" "$ANDROID_SDK_ROOT/cmdline-tools/latest"
''')


In [ ]:
# STEP 4: install Android SDK packages and write env
sh(r'''
set -euxo pipefail
export ANDROID_SDK_ROOT=/content/android-sdk
yes | "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" --sdk_root="$ANDROID_SDK_ROOT" --licenses
yes | "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" --sdk_root="$ANDROID_SDK_ROOT"   "platform-tools" "platforms;android-34" "build-tools;34.0.0"

cat > /content/tgweb_env.sh <<'ENVEOF'
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export ANDROID_SDK_ROOT=/content/android-sdk
export ANDROID_HOME=$ANDROID_SDK_ROOT
export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
ENVEOF
''')


In [ ]:
# STEP 5: optional upload of local files (if needed)
# Optional files: google-services.json, keystore.jks, keystore.properties
from google.colab import files
files.upload()


In [ ]:
# STEP 6: place optional files into project
sh(r'''
set -euxo pipefail
cd /content/tgwebmobile
source /content/tgweb_env.sh

cat > local.properties <<EOF2
sdk.dir=/content/android-sdk
EOF2

if [ -f /content/google-services.json ]; then cp /content/google-services.json app/google-services.json; fi
if [ -f /content/keystore.jks ]; then cp /content/keystore.jks app/keystore.jks; fi
if [ -f /content/keystore.properties ]; then cp /content/keystore.properties keystore.properties; fi
''')


In [ ]:
# STEP 7 (build only): single build stage
sh(r'''
set -euxo pipefail
source /content/tgweb_env.sh
cd /content/tgwebmobile
./gradlew --no-daemon :app:assembleDebug
''')
